<a href="https://colab.research.google.com/github/womenwhocodedc/python-community/blob/master/IntrotoDataScience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Handling Dirty and Missing Data Using Pandas

Presented by

Sian Lewis, Booz Allen Hamilton, sian.lewis@gmail.com, www.linkedin/a/allsian

Valeria Rozenbaum, Thomson Reuters, valeria.rozenbaum@gmail.com

#### NOTE: OPEN IN PLAYGROUND MODE IF YOU CAN

# ABOUT THE DATASET

**The dataset we're working with today comes from the  [FlourishOA github repo](https://github.com/FlourishOA/Data)**


**[FlourishOA](http://flourishoa.org/) was created by University of Washington's Information School Data Science and Analytics Lab to allow for the discovery of reputable open access publications . 
The data was compiled from a variety of sources, including researchers, web scraping, and the publishers themselves.**

**Our dataset consists the following columns:**

* **issn** international standardized code which identifies journals, magazines, and publications of both print and electronica media
* **journal_name** name of the journal
* **pub_name** name of the publisher
* **is_hybrid**   definition unknown
* **category**  subject classification of the journal
* **url** web link to the publication



In [0]:
## First things first, let's import the packages we'll be using

import pandas as pd # Pandas library for data manipulation 
import numpy as np  # Numpy library for scientific computation 
import io #for allowing pandas.read_csv to read in the file in google colab
import requests # Allows requesting url

####################################### 
#######################################

url = 'https://raw.githubusercontent.com/FlourishOA/Data/master/estimated-article-influence-scores-2015.csv'
df = pd.read_csv(url)


# Part 1: Cleaning Dirty Data 

 **Python References and Documentation**
*  [Python Encodings](https://docs.python.org/3.7/library/codecs.html#encodings-and-unicode) 
*  [Pandas Read CSV](https://docs.python.org/2.4/lib/standard-encodings.html)
* [Regular Expressions](https://docs.python.org/3.4/howto/regex.html)
* [Regular Expressions with Pandas](https://www.dataquest.io/blog/regular-expressions-data-scientists/)

## 1.a: Encoding

Character encodings allow your computer to interpret binary bytes (ex:111010101101) as real characters. This is done by mapping human readable characters to byte strings.

While there are many encodings, UTF-8 is one of the most commonly used and is often the default for Python.

Because of this, our encoding problems don't become problems until we have non UTF-8 data.  

**General Terms**
- Decoding - To convert a string of bytes to a unicode string aka human readable.
- Encoding - To represent a unicode string as a string of bytes.




In [15]:
# We don't have to specify the encoding when reading in a csv using pandas and most of the time there aren't any issues because the data is likely utf8 encoded
# Spoiler alert: this data is not

this_will_fail = pd.read_csv('https://raw.githubusercontent.com/FlourishOA/Data/master/api_journal11-13-17.csv')

UnicodeDecodeError: ignored

In [27]:
#Lets try reading in the actual csv file we uploaded at the top and specify decoding to utf8
s = requests.get(url).content
utf8_decode = pd.read_csv(io.StringIO(s.decode('utf-8')))
utf8_jn_list = utf8_decode['journal_name'].unique().tolist()


utf8_jn_list[:500]  

['3d research',
 'aaps pharmscitech',
 'abstract and applied analysis',
 'academic psychiatry',
 'academic questions',
 'accreditation and quality assurance',
 'acoustics australia',
 'acrocephalus',
 'acta adriatica',
 'acta agriculturae slovenica',
 'acta amazonica',
 'acta applicandae mathematicae',
 'acta biochimica polonica',
 'acta bioethica',
 'acta biologica cracoviensia series botanica',
 'acta biotheoretica',
 'acta botanica brasilica',
 'acta botanica croatica',
 'acta chimica sinica',
 'acta chimica slovenica',
 'acta cirurgica brasileira',
 'acta clinica croatica',
 'acta diabetologica',
 'acta endoscopica',
 'acta ethologica',
 'acta geophysica',
 'acta geotechnica',
 'acta herpetologica',
 'acta histochemica et cytochemica',
 'acta ichthyologica et piscatoria',
 'acta informatica',
 'acta limnologica brasiliensia',
 'acta mechanica',
 'acta mechanica sinica',
 'acta medica okayama',
 'acta medica portuguesa',
 'acta montanistica slovaca',
 'acta neurobiologiae experiment

**We are able to read the file in. Sometimes data will have the "?" character. If that is the case, we must do a different type of encoding.**

---



**The ? signifies an unknown character and means that there is no utf8 mapping for that particular byte to be translated.**

In [0]:
# Let's try using a different encoding 
# Fun fact...kind of: latin1 is an alias for ISO-8859-1 

df = pd.read_csv('https://raw.githubusercontent.com/FlourishOA/Data/master/api_journal11-13-17.csv',encoding='ISO-8859-1') 
jn_list = df['journal_name'].unique().tolist()


jn_list[:500]

#### With the correct encoding, we can read the text from our previous example and see what the unknown characters were



*   utf8:

Onom?zein : Revista De Ling??stica, Filolog?a Y Traducci?nOnomázein
*   ISO-8859-1:        

Onomázein : Revista De Lingüística, Filología Y Traducción



In [29]:
## Let's do a quick profile of our dataset 

def eda(dataframe):
    print('MISSING VALUES\n', dataframe.isnull().sum())
    print('\n DATA TYPES \n', dataframe.dtypes)
    print('\n DATA SHAPE \n', dataframe.shape)
    print('\n DATA DESCRIBE \n', dataframe.describe())
    for item in dataframe:
        print('\n UNIQUE VALUE TOTALS \n',item)
        print(dataframe[item].nunique())
        
eda(df)

MISSING VALUES
 issn               0
journal_name       0
pub_name        1470
is_hybrid          0
category        6331
url             6788
dtype: int64

 DATA TYPES 
 issn            object
journal_name    object
pub_name        object
is_hybrid        int64
category        object
url             object
dtype: object

 DATA SHAPE 
 (13149, 6)

 DATA DESCRIBE 
           is_hybrid
count  13149.000000
mean       0.131949
std        0.338448
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.000000

 UNIQUE VALUE TOTALS 
 issn
13149

 UNIQUE VALUE TOTALS 
 journal_name
11581

 UNIQUE VALUE TOTALS 
 pub_name
3529

 UNIQUE VALUE TOTALS 
 is_hybrid
2

 UNIQUE VALUE TOTALS 
 category
140

 UNIQUE VALUE TOTALS 
 url
6327


## 1.b: Deduplication

In [30]:
# Lets check if we have any serial number duplication


print(any(df['issn'].duplicated())) #any is a method that iterates through strings and returns a boolean value based on the condition. 
                                    #returning false means that no duplicate values were found in issn

False


In [31]:
#We can also check for duplication in multiple columns using the same logic

print(any(df.duplicated(['journal_name', 'pub_name','url']))) 

True


In [32]:
#Lets see the dupes

df[df.duplicated(['journal_name', 'pub_name','url'], keep='first')] #specifying keep=first counts the first occurring value as original and the remaining as duplicates

#Looks like we ran into a problem with identifying actual dupe values.. because nulls

,issn,journal_name,pub_name,is_hybrid,category,url
132,0053-0016,International Journal of Recent Surgical and M...,NaN,0,NaN,NaN
133,0053-0017,International Journal of Recent Surgical and M...,NaN,0,NaN,NaN
134,0053-0018,International Journal of Recent Surgical and M...,NaN,0,NaN,NaN
135,0053-0019,International Journal of Recent Surgical and M...,NaN,0,NaN,NaN
136,0053-0020,International Journal of Recent Surgical and M...,NaN,0,NaN,NaN
137,0053-0021,International Journal of Recent Surgical and M...,NaN,0,NaN,NaN
138,0053-0022,International Journal of Recent Surgical and M...,NaN,0,NaN,NaN
139,0053-0023,International Journal of Recent Surgical and M...,NaN,0,NaN,NaN
140,0053-0024,International Journal of Recent Surgical and M...,NaN,0,NaN,NaN
141,0053-0025,International Journal of Recent Surgical and M...,NaN,0,NaN,NaN


In [0]:
#We don't want to do anything about the nulls just yet. However, they are being counted towards our duplicate values.
#To work around this, we can do a groupby and size on our duplicated columns and add an occurrence count column.


#Let's make it a new dataframe for better access
jour_pub_url_group = df.groupby(['journal_name','pub_name','url']).size().reset_index().\
                    rename(columns={0:'occurrence_count'})

In [46]:
# Lets see how many values are duplicated across journal_name,pub_name and url


jour_pub_url_group[jour_pub_url_group['occurrence_count']>1]

,journal_name,pub_name,url,occurrence_count
749,Biodiversity and Natural History,Centro de Estudios en Biodiversidad,http://www.biodiversnathist.com,2


In [0]:
#Now let's see how many values are duplicated in the entire dataset

dupe_check_all_df= df.groupby(df.columns.tolist()).size().reset_index().\
                       rename(columns={0:'occurrence_count'})

In [48]:
# To get the total number of dupes, we can subtract the shape of our data from the occurence count total

dupe_check_all_df['occurrence_count'].sum() - dupe_check_all_df.shape[0]

0

There is a duplicate value in the journal_name, pub_name, and url columns. However, I am in no way  a subject matter expert in academic journals, so  I don't want to assume that this means that the data is inherently wrong. 
Since the issn column can be considered a unique identifier and does not contain duplicates, I believe the right move is to continue working with the data as is and consult the owners of the dataset later for clarification.

## 1.c:  Standardization and String Matching

In [49]:
# From what we've already seen of the data, we can recognize that character case is inconsistent in pub_name and category columns
# This can be quickly fixed by converting all columns with object datatypes to lowercase

df = df.apply(lambda x: x.str.lower() if(x.dtype == 'object') else x)

df.head(2)

,issn,journal_name,pub_name,is_hybrid,category,url
0,0001-527x,acta biochimica polonica,acta biochimica polonica,0,molecular and cell biology,NaN
1,0002-0397,africa spectrum,NaN,0,NaN,NaN


In [50]:
#We can alse apply the same string functions to individual columns...


df['journal_name'] = df['journal_name'].str.title() # Let's make journal_name title case
                                                    # Careful with this one,it doesn't discriminate and will capitalize articles and prepositions

df['issn']= df['issn'].str.upper() #And lets make issn uppercase 

df.head()

,issn,journal_name,pub_name,is_hybrid,category,url
0,0001-527X,Acta Biochimica Polonica,acta biochimica polonica,0,molecular and cell biology,NaN
1,0002-0397,Africa Spectrum,NaN,0,NaN,NaN
2,0003-0090,Bulletin Of The American Museum Of Natural His...,amer museum natural history,0,ecology and evolution,NaN
3,0003-5521,L'Anthropologie,elsevier,1,anthropology,NaN
4,0004-1254,Arhiv Za Higijenu Rada I Toksikologiju-Archive...,NaN,0,NaN,NaN


In [51]:
 df['category'].unique()

# Looking at the unique values of the category column we can see that we still have some problems with inconsistent delimiter use
# We want to standardize the data and make it more consistent

array(['molecular and cell biology', nan, 'ecology and evolution',
       'anthropology', 'robotics', 'physics and chemistry', 'psychology',
       'medicine', 'neuroscience', 'veterinary', 'economics', 'law',
       'gastroenterology', 'plant biology', 'orthopedics', 'agriculture',
       'linguistics', 'history and philosophy of science', 'mathematics',
       'oncology', 'sports medicine', 'food science', 'psychiatry',
       'science', 'radiology', 'urology', 'environmental health',
       'social sciences', 'ophthalmology', 'marketing',
       'astronomy and astrophysics', 'operations research',
       'infectious diseases', 'literary studies', 'plastic surgery',
       'energy', 'structural engineering',
       'philosophy. psychology. religion', 'fine arts',
       'music and books on music', 'history america',
       'geography. anthropology. recreation', 'political science',
       'language and literature', 'technology', 'education',
       'general works', 'history (general)

In [0]:
# Lets replace 'and' with '&'
df['category'].replace(to_replace='and', value='&', regex=True,inplace=True) 

In [53]:
# Now back to those pesky delimiters ...

# Overall, there are 12 metacharacters you should look out for  . ^ $ * + ? { } [ ] \ | ( )
#Both the pipe(|) and period(.) are considered metacharacters because they have special meaning.
 
# Regular expressions use the backslash character ('\') to indicate special forms or to allow special characters to be used without invoking their meaning. 

# This conflicts with Python’s usage of the same character for the same purpose in string literals and can become a pain when backslash
# proliferation makes things unreadable

print('ESCAPING WITH BACKSLASHES\n')
print('ROW CONTAINS PIPE \n', df['category'].str.contains('\\|').sum())
print('ROW CONTAINS PERIOD \n', df['category'].str.contains('\\.').sum())
print('ROW CONTAINS BACKSLASH \n', df['category'].str.contains('\\\\').sum()) #This is when the backslashes really scale up
print('ROW CONTAINS OPEN PARENTHESIS \n', df['category'].str.contains('\\(').sum())
print('ROW CONTAINS CLOSE PARENTHESIS \n', df['category'].str.contains('\\)').sum())

ESCAPING WITH BACKSLASHES

ROW CONTAINS PIPE 
 102
ROW CONTAINS PERIOD 
 595
ROW CONTAINS BACKSLASH 
 0
ROW CONTAINS OPEN PARENTHESIS 
 128
ROW CONTAINS CLOSE PARENTHESIS 
 128


In [54]:
# A cleaner solution for this is to use raw string notation in addition to a backslash by adding an r outside of the quoted expression.
print('ESCAPING WITH RAW STRING NOTATION \n')
print('ROW CONTAINS PIPE \n', df['category'].str.contains(r'\|').sum())
print('ROW CONTAINS PERIOD \n', df['category'].str.contains(r'\.').sum())
print('ROW CONTAINS BACKSLASH \n', df['category'].str.contains(r'\\').sum())
print('ROW CONTAINS OPEN PARENTHESIS \n', df['category'].str.contains(r'\(').sum())
print('ROW CONTAINS CLOSE PARENTHESIS \n', df['category'].str.contains(r'\)').sum())

ESCAPING WITH RAW STRING NOTATION 

ROW CONTAINS PIPE 
 102
ROW CONTAINS PERIOD 
 595
ROW CONTAINS BACKSLASH 
 0
ROW CONTAINS OPEN PARENTHESIS 
 128
ROW CONTAINS CLOSE PARENTHESIS 
 128


In [55]:
#Intuitively, we can see that parenthesis are being used differently than pipes in the context of this dataset. 
#Because of this, we will leave them out of the special chars replacement.


special_chars=r'\.|\^|\$|\*|\+|\?|\{|\}|\[|\]|\\|\|' # Note that with the backslash, we're searching for the literal | character 
                                                     # Without the backslash we're using | as an OR statement

  
len(df[df['category'].str.contains(special_chars,na=False,regex=True)])

684

In [0]:
#Lets replace the special characters with an underscore.

df['category'].replace(to_replace=special_chars, value='_', regex=True, inplace=True) 

In [57]:
df.category

0        molecular & cell biology
1                             NaN
2             ecology & evolution
3                    anthropology
4                             NaN
5                             NaN
6                        robotics
7                             NaN
8                             NaN
9             physics & chemistry
10                     psychology
11                            NaN
12                       medicine
13                   neuroscience
14                            NaN
15                     veterinary
16            physics & chemistry
17                       medicine
18                            NaN
19                       medicine
20                            NaN
21                      economics
22                            NaN
23                            NaN
24                            law
25                            NaN
26                            NaN
27               gastroenterology
28                            NaN
29            

In [0]:
#Now lets clean up the whitespace before and after the underscore

df['category']=df['category'].replace('_ ', '_',regex=True).replace(' _','_',regex=True)

In [59]:
df['category'].unique()

array(['molecular & cell biology', nan, 'ecology & evolution',
       'anthropology', 'robotics', 'physics & chemistry', 'psychology',
       'medicine', 'neuroscience', 'veterinary', 'economics', 'law',
       'gastroenterology', 'plant biology', 'orthopedics', 'agriculture',
       'linguistics', 'history & philosophy of science', 'mathematics',
       'oncology', 'sports medicine', 'food science', 'psychiatry',
       'science', 'radiology', 'urology', 'environmental health',
       'social sciences', 'ophthalmology', 'marketing',
       'astronomy & astrophysics', 'operations research',
       'infectious diseases', 'literary studies', 'plastic surgery',
       'energy', 'structural engineering',
       'philosophy_psychology_religion', 'fine arts',
       'music & books on music', 'history america',
       'geography_anthropology_recreation', 'political science',
       'language & literature', 'technology', 'education',
       'general works', 'history (general) & history of euro

# Part 2: Handling Missing Data

## 2.1 On Missingness 

Resource: http://www.stat.columbia.edu/~gelman/arm/missing.pdf


1. **Missingness completely at random (MCAR)**. A variable is missing completely at random if the probability of missingness is the same for all units, for example, if each survey respondent decides whether to answer the “earnings” question by rolling a die and refusing to answer if a “6” shows up. If data are missing completely at
random, then throwing out cases with missing data does not bias your inferences.

2. **Missingness at random (MAR)**. Missing at random means that the propensity for a data point to be missing is not related to the missing data, but it is related to some of the observed data.
Most missingness is not completely at random, as can be seen from the data themselves. A more general assumption, missing at random, is that the probability a variable
is missing depends only on available information. Thus, if sex, race, education, and age are recorded for all the people in the survey, then “earnings” is missing at random if the probability of nonresponse to this question depends only on these other, fully recorded variables. It is often reasonable to model this process as a logistic regression, where the outcome variable equals 1 for observed cases
and 0 for missing. When an outcome variable is missing at random, it is acceptable to exclude the missing cases (that is, to treat them as NA’s), as long as the regression controls for all the variables that affect the probability of missingness. Thus, any model
for earnings would have to include predictors for ethnicity, to avoid nonresponse bias. This missing-at-random assumption (a more formal version of which is sometimes called the ignorability assumption) in the missing-data framework is the basically same sort of assumption as ignorability in the causal framework. Both require that sufficient information has been collected that we can “ignore” the
assignment mechanism (assignment to treatment, assignment to nonresponse).
3. **Missingness not at random (MNAR)**: 
. Missingness is no longer “at random” if it depends on information that has not been recorded and this information also predicts the missing values. For example, suppose that “surly”
people are less likely to respond to the earnings question, surliness is predictive
of earnings, and “surliness” is unobserved. Or, suppose that people with college
degrees are less likely to reveal their earnings, having a college degree is predictive of earnings, and there is also some nonresponse to the education question.
Then, once again, earnings are not missing at random. If missingness is not at random, it must be explicitly modeled, or else you must
accept some bias in your inferences. There are two types:

*   **Missingness that depends on unobserved predictors**. A familiar example from medical studies is that if a particular treatment causes
discomfort, a patient is more likely to drop out of the study. This missingness is
not at random (unless “discomfort” is measured and observed for all patients).
*   **Missingness that depends on the missing value itself**. Finally, a particularly difficult situation arises when the probability of missingness depends on the (potentially missing) variable itself. For example, suppose that people with higher earnings are less likely to reveal them. In the extreme case (for example, all persons earning more than $100,000 refuse to respond), this is called censoring, but
even the probabilistic case causes difficulty.

In the first two cases, it is safe to remove the data with missing values depending upon their occurrences. 

In the third case (MNAR), removing observations with missing values can produce a bias in the model. 

So we have to be really careful before removing observations.




##2.2 Choices, Choices

There are 3 choices for missing data:

**1.   Imputation: Fill in the missing data**
*   User provided data
*   Summary statistic (Be careful here: mean, median, mode, etc. Use median if you have outliers)
*   Modeled or imputed data
*   Data present in dataset

**2. Deletion: Drop rows or columns with missing data**
*   Drop all 
*   Drop some, based on conditions or a threshold

**3. Leave it as is**

**When should one fill in data, drop data, or leave it alone? **

It depends. . . .



![Handling Missing Data](https://cdn-images-1.medium.com/max/1600/1*_RA3mCS30Pr0vUxbp25Yxw.png)

##  2.3 Let's Get to Work

### 2.3.1 Data Quality Check
Let's check out our data

In [60]:
#Check out the dataset
df.head()

,issn,journal_name,pub_name,is_hybrid,category,url
0,0001-527X,Acta Biochimica Polonica,acta biochimica polonica,0,molecular & cell biology,NaN
1,0002-0397,Africa Spectrum,NaN,0,NaN,NaN
2,0003-0090,Bulletin Of The American Museum Of Natural His...,amer museum natural history,0,ecology & evolution,NaN
3,0003-5521,L'Anthropologie,elsevier,1,anthropology,NaN
4,0004-1254,Arhiv Za Higijenu Rada I Toksikologiju-Archive...,NaN,0,NaN,NaN


## 2.4 Assessing Missingness

How much missingness is there?

In [61]:
#Check out the column types
df.dtypes

issn            object
journal_name    object
pub_name        object
is_hybrid        int64
category        object
url             object
dtype: object

In [62]:
# Let's find out which columns have missing data
missing_columns = list(df.columns[df.isnull().any()])
missing_columns

['pub_name', 'category', 'url']

In [63]:
#Let's find out how many missing values are in each column

for col in missing_columns:
    num_missing = df[df[col].isnull() == True].shape[0]
    print('number missing for column {}: {}'.format(col, 
                                                    num_missing))

number missing for column pub_name: 1470
number missing for column category: 6331
number missing for column url: 6788


In [64]:
#Percentage of missingness per column
for col in missing_columns:
    percent_missing = df[df[col].isnull() == True].shape[0]  / df.shape[0] * 100
    print('percent missing for column {}: {}%'.format(
        col, percent_missing))

percent missing for column pub_name: 11.179557380789413%
percent missing for column category: 48.148148148148145%
percent missing for column url: 51.62369761959085%


## 2.5 DELETION
If you have a "small" amount of data missing:


1.   **Listwise deletion**: remove ALL data for an observation that has at least 1 missing value. 
> *Problem: Meeting MCAR assumptions are rare; as a result most of time you'll produce biased parameters and estimates*


2.   **Pairwise deletion**: Keep all data that isn't missing. Even though it increases the power of your analysis, you'll end up with different numbers of observations contributing to different parts of your model. This makes interpretation very difficult.

2.   **Deleting Columns**: Keep all data that isn't missing. Even though it increases the power of your analysis, you'll end up with different numbers of observations contributing to different parts of your model. This makes interpretation very difficult.



### 2.5.1 Remove Null Values Upon Reading in Data

In [68]:
#Call na_values to remove n/a values
# Making a list of missing value types
# Read in data using Pandas and quickly ELIMINATE missing values based on list of missing value types
missing_values = ["n/a", "na", "--"]
df = pd.read_csv(url, na_values = missing_values)
df.head(10)

,Unnamed: 0,journal_name,issn,citation_count_sum,paper_count_sum,avg_cites_per_paper,proj_ai,proj_ai_year
0,0,3d research,2092-6731,151.0,106.0,1.424528,0.290,2015
1,1,aaps pharmscitech,1530-9932,2208.0,801.0,2.756554,0.665,2015
2,2,abstract and applied analysis,1687-0409,3005.0,2923.0,1.028053,0.192,2015
3,3,academic psychiatry,1545-7230,537.0,490.0,1.095918,0.208,2015
4,4,academic questions,1936-4709,40.0,67.0,0.597015,0.097,2015
5,5,accreditation and quality assurance,1432-0517,255.0,331.0,0.770393,0.134,2015
6,6,acoustics australia,1839-2571,30.0,25.0,1.200000,0.234,2015
7,7,acrocephalus,2199-6067,9.0,15.0,0.600000,0.098,2015
8,8,acta adriatica,1846-0453,28.0,37.0,0.756757,0.131,2015
9,9,acta agriculturae slovenica,1854-1941,71.0,97.0,0.731959,0.125,2015


In [67]:
df.shape

(3615, 8)

## 2.6 What to do with NaNs?


If you need to fill in errors or blanks, use the ```fillna()``` and ```dropna()``` methods. 

It seems quick, but all manipulations of the data should be documented so you can explain them to someone at a later time.

You could fill the NaNs with strings, or if they are numbers you could use the mean or the median value. There is a lot of debate on what do with missing or malformed data, and the correct answer is … it depends.

You’ll have to use your best judgement and input from the people you’re working with on why removing or filling the data is the best approach.



In [69]:
#Now we see that 6788 values are filled in with 0
df.url.value_counts()

AttributeError: ignored

### 2.6.1 Filling in Data

In [0]:
# First, find the value counts of the URL column
df.url.value_counts()


https://www.springer.com/us/open-access/springer-open-choice                     14
https://www.hindawi.com/journals/acp/                                             3
http://www.degruyter.com/view/j/plua                                              2
http://www.ddtjournal.org/                                                        2
http://www.biogeosciences.net                                                     2
http://www.springer.com/us/open-access/springer-open-choice                       2
https://www.hindawi.com/journals/jqre/                                            2
http://epress.lib.uts.edu.au/journals/index.php/AJCEB                             2
http://www.kmuj.kmu.edu.pk/index                                                  2
http://newvoices.org.au/                                                          2
http://www.collabra.org                                                           2
http://www.egms.de/dynamic/en/journals/zma/index.htm                        

In [0]:
# The top missing values in the URL column
df.url.value_counts().head()


https://www.springer.com/us/open-access/springer-open-choice    14
https://www.hindawi.com/journals/acp/                            3
http://www.degruyter.com/view/j/plua                             2
http://www.ddtjournal.org/                                       2
http://www.biogeosciences.net                                    2
Name: url, dtype: int64

In [0]:
# Fill in missing data using the .fillna method
# Let's fill in missing values with 0
df.url = df.url.fillna(0)

# Fill NaN with ' '
#df['url'] = df['url'].fillna(' ')

# Fill NaN with 999
#df['url'] = df['url'].fillna(999)

# Fill NaN with the mean of the column -- for numeric columns only!!
#df['url'] = df['url'].fillna(df['url'].mean())

You can also propagate non-null values forward or backwards by using ```method=’pad’``` as the method argument. 


It will fill the next value in the dataframe with the previous non-NaN value. 

When you want to fill one value, use ```limit=1```.

If you don't set a limit, you'll fill in the entire dataframe 

Let's create a dataframe to demo forward fill and backfilll:


In [0]:
missing_df = pd.DataFrame(data={'col1':[np.nan, np.nan, 23,3,14,5,13, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]})
missing_df

,col1
0,NaN
1,NaN
2,23.0
3,3.0
4,14.0
5,5.0
6,13.0
7,NaN
8,NaN
9,NaN


In [2]:
missing_df.fillna(method='pad', limit=1)
# See that index 7 is filled in with index 6's value  

NameError: ignored

In [1]:
# If you don't set a limit, you'll fill in the rest of the entire dataframe 
missing_df.fillna(method='pad')

NameError: ignored

#### 2.6.2 Backfilling

We are not limited to forward filling. We can  also backfill with bfill.

In [0]:
#The first two values are NaN's. Let's backfill them
missing_df

,col1
0,NaN
1,NaN
2,23.0
3,3.0
4,14.0
5,5.0
6,13.0
7,NaN
8,NaN
9,NaN


In [0]:
# Fill the first two NaN values with the first available value
missing_df.fillna(method='bfill')

,col1
0,23.0
1,23.0
2,23.0
3,3.0
4,14.0
5,5.0
6,13.0
7,NaN
8,NaN
9,NaN


##2.7 Dropping Missing Data

Use the ```dropna``` method to drop rows or columns with NaNs. 

In [0]:
# Drop any rows which have any nans
missing_df.dropna()


,col1
2,23.0
3,3.0
4,14.0
5,5.0
6,13.0


In [0]:
# Drop columns that have any nans
missing_df.dropna(axis=1)


""
0
1
2
3
4
5
6
7
8
9


###2.7.1 NaNs thresholding
The parameter thresh=N requires that a column has at least N non-NaNs to survive. Think of this as the lower limit for missing data you will find acceptable in your columns.

In [0]:
#Use a threshold to drop NaNs
# Only drop columns which have at least 20% non-NaNs
missing_df.dropna(thresh=int(missing_df.shape[0] * .20), axis=1)

,col1
0,NaN
1,NaN
2,23.0
3,3.0
4,14.0
5,5.0
6,13.0
7,NaN
8,NaN
9,NaN


## 2.8 IMPUTATION

###2.8.1 SIMPLE IMPUTATION

Replace missing values, encoded as ```np.nan```, using the mean value of the columns (axis 0) that contain the missing values.

Missing values can be imputed with a provided constant value, or using the statistics (mean, median or most frequent) of each column in which the missing values are located.

####2.8.1.1 For Numeric NaNs

In [0]:
#NUMERIC NaNs
#Impute with the mean
#This will look for all columns where we have NaN value and replace the NaN value with specified test statistic.

from sklearn.impute import SimpleImputer
imputed_data = SimpleImputer(missing_values=np.nan, strategy='mean')
#Change strategy to 'median', and for the mode: 'most_frequent' 

imputed_data.fit(missing_df) #use .fit

missing_df = pd.DataFrame(imputed_data.transform(missing_df)) #use.transform
missing_df


,0
0,11.6
1,11.6
2,23.0
3,3.0
4,14.0
5,5.0
6,13.0
7,11.6
8,11.6
9,11.6


In [0]:
#Let's look at the parameters of the SimpleImputer
imputed_data

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='mean', verbose=0)

####2.8.1.2 For Categorical or *String* Data

The SimpleImputer class also supports categorical data represented as string values or pandas categoricals when using the 'most_frequent' or 'constant' strategy.

In [0]:
#Let's create a dataframe with categorical or string data
categorical_df = pd.DataFrame([["a", "x"],
                               [np.nan, "y"],
                               ["a", np.nan],
                               ["b", "y"]], dtype="category")

#Let's take a look at this new dataframe
categorical_df

#the most frequent value in each column is 'a' and the 'y'

,0,1
0,a,x
1,NaN,y
2,a,NaN
3,b,y


In [0]:
#Replacing the NaNs with the most frequent observation
cat_imp = SimpleImputer(strategy="most_frequent")
print(cat_imp.fit_transform(categorical_df))      

[['a' 'x']
 ['a' 'y']
 ['a' 'y']
 ['b' 'y']]


In [0]:

cat_imp

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='most_frequent', verbose=0)

## 2.9 Now Let's Get Fancy: FancyImpute

####2.9.1 Imputation Algorithms

**SimpleFill**: Replaces missing entries with the mean or median of each column.

**KNN**: Nearest neighbor imputations which weights samples using the mean squared difference on features for which two rows both have observed data.

**IterativeImputer**: A strategy for imputing missing values by modeling each feature with missing values as a function of other features in a round-robin fashion.

**SoftImpute**: Matrix completion by iterative soft thresholding of SVD decompositions. Inspired by the softImpute package for R, which is based on Spectral Regularization Algorithms for Learning Large Incomplete Matrices by Mazumder et. al.


**MatrixFactorization**: Direct factorization of the incomplete matrix into low-rank U and V, with an L1 sparsity penalty on the elements of U and an L2 penalty on the elements of V. Solved by gradient descent.

**NuclearNormMinimization**: Simple implementation of Exact Matrix Completion via Convex Optimization by Emmanuel Candes and Benjamin Recht using cvxpy. Too slow for large matrices.

**BiScaler**: Iterative estimation of row/column means and standard deviations to get doubly normalized matrix. Not guaranteed to converge but works well in practice. Taken from Matrix Completion and Low-Rank SVD via Fast Alternating Least Squares.

### Implementing Multiple Imputation
We are using the metric -- Mean Square Error.

Let's run all the imputation algorithms to see which one gives us the lowest error. The goal is to have an MSE of zero. 

In [0]:
#Let's run all the imputation algorithms and see which ones give us the lowest error

import numpy as np
from fancyimpute import (
    BiScaler,
    KNN,
    IterativeImputer,
    NuclearNormMinimization,
    SoftImpute,
    SimpleFill)

n = 200
m = 20
inner_rank = 4
X = np.dot(np.random.randn(n, inner_rank), np.random.randn(inner_rank, m))
print("Mean squared element: %0.4f" % (X ** 2).mean())

# X is a data matrix which we're going to randomly drop entries from
missing_mask = np.random.rand(*X.shape) < 0.1
X_incomplete = X.copy()
# missing entries indicated with NaN
X_incomplete[missing_mask] = np.nan

meanFill = SimpleFill("mean")
X_filled_mean = meanFill.fit_transform(X_incomplete)

# Model each feature with missing values as a function of other features, and
# use that estimate for imputation.
X_filled_ii = IterativeImputer().fit_transform(X_incomplete)

# Use 3 nearest rows which have a feature to fill in each row's missing features
knnImpute = KNN(k=3)
X_filled_knn = knnImpute.fit_transform(X_incomplete)

# matrix completion using convex optimization to find low-rank solution
# that still matches observed values. Slow!
X_filled_nnm = NuclearNormMinimization().fit_transform(X_incomplete)

# Instead of solving the nuclear norm objective directly, instead
# induce sparsity using singular value thresholding
softImpute = SoftImpute()

# simultaneously normalizes the rows and columns of your observed data,
# sometimes useful for low-rank imputation methods
biscaler = BiScaler()

# rescale both rows and columns to have zero mean and unit variance
X_incomplete_normalized = biscaler.fit_transform(X_incomplete)

X_filled_softimpute_normalized = softImpute.fit_transform(X_incomplete_normalized)
X_filled_softimpute = biscaler.inverse_transform(X_filled_softimpute_normalized)

X_filled_softimpute_no_biscale = softImpute.fit_transform(X_incomplete)

meanfill_mse = ((X_filled_mean[missing_mask] - X[missing_mask]) ** 2).mean()
print("meanFill MSE: %f" % meanfill_mse)

# print mean squared error for the four imputation methods above
# print mean squared error for the three imputation methods above
ii_mse = ((X_filled_ii[missing_mask] - X[missing_mask]) ** 2).mean()
print("Iterative Imputer norm minimization MSE: %f" % ii_mse)

nnm_mse = ((X_filled_nnm[missing_mask] - X[missing_mask]) ** 2).mean()
print("Nuclear norm minimization MSE: %f" % nnm_mse)

softImpute_mse = ((X_filled_softimpute[missing_mask] - X[missing_mask]) ** 2).mean()
print("SoftImpute MSE: %f" % softImpute_mse)

softImpute_no_biscale_mse = (
    (X_filled_softimpute_no_biscale[missing_mask] - X[missing_mask]) ** 2).mean()
print("SoftImpute without BiScale MSE: %f" % softImpute_no_biscale_mse)


knn_mse = ((X_filled_knn[missing_mask] - X[missing_mask]) ** 2).mean()
print("knnImpute MSE: %f" % knn_mse)

Mean squared element: 4.4201
Imputing row 1/200 with 1 missing, elapsed time: 0.012
Imputing row 101/200 with 0 missing, elapsed time: 0.018
----------------------------------------------------------------------------
	SCS v2.1.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 51532
eps = 1.00e-05, alpha = 1.50, max_iters = 50000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-03
Variables n = 32310, constraints m = 40310
Cones:	primal zero / dual free vars: 4000
	linear vars: 12000
	sd vars: 24310, sd blks: 1
Setup time: 7.11e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 2.25e+21  2.08e+21  1.00e+00 -2.13e+24  4.35e+23  7.23e+23  

### RESULTS
We see that the Iterative Imputer and the Nuclear norm minimuxation has an MSE of 0.

We see that simply filling missing values with the mean (meanFill) results in exponentially larger error. 

For the love of all that is good in the world, use iterative imputation over simple methods (where appropriate)